Projektarbeit Data Science

Erkundung und Vorhersage der Hauspreise für Boston

Willi Brügel

In [ ]:
# ----------------------------------------
# 1a) Datensatz importieren, fehlende 
# und doppelte Werte prüfen und behandeln
# ----------------------------------------

import pandas as pd
import numpy as np

# URL Datensatz
data_url = "http://lib.stat.cmu.edu/datasets/boston"
# erste Zeilen ausblenden, nur Überschriften
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

# Features zusammenbauen (13 Spalten)
# Daten aus geraden und ungeraden Zeilen zusammenführen
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])

# Zielvariable extrahieren
target = raw_df.values[1::2, 2]
target = pd.DataFrame(target, columns=["MEDV"])

# Benennung der Spalten
columns = [
    "CRIM","ZN","INDUS","CHAS","NOX","RM","AGE",
    "DIS","RAD","TAX","PTRATIO","B","LSTAT"
]

# Dataframe erzeugen
df = pd.DataFrame(data, columns=columns)
# target hinzu
df["MEDV"] = target

#print(df)

# doppelte Werte löschen 
# (im Datensatz sind keine doppelten Werte vorhanden, 
# wie mit den folgenden Zeilen geprüft wurde)
# len_vor_dropna = len(df)
new_df = df.dropna()
# len_nach_dropna = len(df)
# print(len_vor_dropna - len_nach_dropna)

# mit folgendem Code wurde geprüft, ob es leere Zellen gibt
# leere_zellen = df.isna().sum().sum()
# print(leere_zellen)
# --> Ergebnis: keine leeren Zellen

# =============================================================================
# Bedeutung der Variablen im Boston Housing Datensatz
# =============================================================================
# CRIM     Kriminalitätsrate pro Kopf in der Stadt
# ZN       Anteil des Wohngebietes mit Grundstücken über 25.000 ft² (~2322 m²)
# INDUS    Anteil der Gewerbefläche (ohne Einzelhandel) in der Stadt
# CHAS     Dummy-Variable: 1 = grenzt an den Charles River, sonst 0
# NOX      Konzentration von Stickoxiden (in Teilen pro 10 Millionen)
# RM       Durchschnittliche Anzahl der Zimmer pro Wohneinheit
# AGE      Anteil der vor 1940 erbauten, selbst genutzten Häuser
# DIS      Gewichtete Entfernung zu fünf Arbeitszentren in Boston
# RAD      Index der Erreichbarkeit über Autobahnen
# TAX      Grundsteuer pro 10.000 USD Immobilienwert
# PTRATIO  Verhältnis von Schülern zu Lehrkräften in der Stadt
# B        1000 * (Bk – 0.63)², wobei Bk = Anteil der schwarzen Bevölkerung
# LSTAT    Prozentualer Anteil der Bevölkerung mit niedrigem sozialen Status
# MEDV     Medianwert der selbstgenutzten Häuser (in 1.000 USD) → Zielvariable
# =============================================================================



        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0    0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1    0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2    0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3    0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4    0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...    ...   
501  0.06263   0.0  11.93   0.0  0.573  6.593  69.1  2.4786  1.0  273.0   
502  0.04527   0.0  11.93   0.0  0.573  6.120  76.7  2.2875  1.0  273.0   
503  0.06076   0.0  11.93   0.0  0.573  6.976  91.0  2.1675  1.0  273.0   
504  0.10959   0.0  11.93   0.0  0.573  6.794  89.3  2.3889  1.0  273.0   
505  0.04741   0.0  11.93   0.0  0.573  6.030  80.8  2.5050  1.0  273.0   

     PTRATIO       B  LSTAT  MEDV  
0       15.3  396.90   4.98  24.0  
1       17.8  396.90   9.14

In [ ]:
# ----------------------------------------
# 1b) Analyse der Eigenschaften: 
# Mittelwert, Modus, Median, Varianz, Abweichung
# ----------------------------------------

statistic_table = pd.DataFrame({
    "Mittelwert": df.mean(),
    "Median": df.median(),
    "Varianz": df.var(),
    "Standardabweichung": df.std(),
    "Minimum": df.min(),
    "Maximum": df.max(),
})

print(statistic_table)


         Mittelwert     Median  Standardabweichung       Varianz    Minimum  \
CRIM       3.613524    0.25651            8.601545     73.986578    0.00632   
ZN        11.363636    0.00000           23.322453    543.936814    0.00000   
INDUS     11.136779    9.69000            6.860353     47.064442    0.46000   
CHAS       0.069170    0.00000            0.253994      0.064513    0.00000   
NOX        0.554695    0.53800            0.115878      0.013428    0.38500   
RM         6.284634    6.20850            0.702617      0.493671    3.56100   
AGE       68.574901   77.50000           28.148861    792.358399    2.90000   
DIS        3.795043    3.20745            2.105710      4.434015    1.12960   
RAD        9.549407    5.00000            8.707259     75.816366    1.00000   
TAX      408.237154  330.00000          168.537116  28404.759488  187.00000   
PTRATIO   18.455534   19.05000            2.164946      4.686989   12.60000   
B        356.674032  391.44000           91.294864  